In [68]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import time as t
from sklearn import preprocessing as pre

path="/mnt/D210C36810C35261/dev/telstra/data/"
#path="C:\\dev\\telstra\\data\\"

trainFile="train.csv"
testFile="test.csv"
resourceFile="resource_type.csv"
eventTypeFile="event_type.csv"
logFeatureFile="log_feature.csv"
severityTypeFile="severity_type.csv"

test=pd.read_csv(filepath_or_buffer=path+testFile,delimiter=",",header=0)
train=pd.read_csv(filepath_or_buffer=path+trainFile,delimiter=",",header=0)

resource=pd.read_csv(filepath_or_buffer=path+resourceFile,delimiter=",",header=0)
event=pd.read_csv(filepath_or_buffer=path+eventTypeFile,delimiter=",",header=0)
feature=pd.read_csv(filepath_or_buffer=path+logFeatureFile,delimiter=",",header=0)
severity=pd.read_csv(filepath_or_buffer=path+severityTypeFile,delimiter=",",header=0)
join=pd.DataFrame({'id':[]})
datasets={'train':train,'test':test,'resource':resource,'event':event,'feature':feature,'severity':severity}
notCat=['id','volume','prior']

#creates join dataset table that will store features for train and test datasets
datasets['train']['sample']='train'
datasets['test']['sample']='test'
datasets['test']['fault_severity']=np.nan
join=pd.concat([datasets['train'],datasets['test']],ignore_index=True)

In [53]:
join.head()

,fault_severity,id,location,sample
0,1,14121,location 118,train
1,0,9320,location 91,train
2,1,14394,location 152,train
3,1,8218,location 931,train
4,0,14804,location 120,train


In [70]:
dfp=train.groupby('location').mean() #add 2*std
dfp['location']=dfp.index
dfp=dfp.reindex(columns=['fault_severity','location'])
dfp.columns=['prior','location']
dfp['prior']=pre.minmax_scale(dfp['prior'],feature_range=(0,2))
join=pd.merge(join,dfp,on='location')

In [71]:
dfw=train.groupby('location').count()
dfw['location']=dfw.index
dfw=dfw.reindex(columns=['fault_severity','location'])
dfw.columns=['weight','location']
dfw['weight']=pre.minmax_scale(dfw['weight'],feature_range=(0,1))
join=pd.merge(join,dfw,on='location')
join.head()

/usr/share/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)


,fault_severity,id,location,sample,prior,weight
0,1,14121,location 118,train,0.090909,0.380952
1,0,6821,location 118,train,0.090909,0.380952
2,0,12008,location 118,train,0.090909,0.380952
3,0,18441,location 118,train,0.090909,0.380952
4,0,9479,location 118,train,0.090909,0.380952


fault_severity    float64
id                  int64
location           object
sample             object
prior             float64
weight            float64
dtype: object

In [55]:
pd.crosstab(train['location'],train['fault_severity'])

fault_severity,0,1,2
location,,,
location 1,14,8,2
location 10,1,0,0
location 100,13,2,0
location 1000,3,0,0
location 1002,1,0,0
location 1005,1,0,0
location 1006,3,0,0
location 1007,16,3,0
location 1008,34,21,16


In [ ]:
#joins features from different datasets into one dataframe, features only for evaluated ids are extracted
#(i.e. some features may be not related to any id)
samples=['train','test']
for key, dataset in datasets.items():
    if key not in samples:
        join=pd.merge(join,dataset,on='id',how='left') #can use concat?

#convert table of ids and classes to binary row feature vectors for each unique id
for key, dataset in datasets.items():
    tmp=pd.get_dummies(dataset,dummy_na=True)
    tmp=tmp.groupby('id').sum()
    tmp['id']=tmp.index
    dataset=tmp
    datasets[key]=dataset

join=join.set_index(['sample',join.index])

In [12]:
#normalizes a continuous feature
join['volume']=pre.scale(join['volume'].astype(float))

#fills in missing data
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].median() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)
    
join = DataFrameImputer().fit_transform(join)
#join=join.fillna(value=0)

#creates features and targets for final training
features=join.columns.values.tolist()
non_features=[]
for x in features:
    if "_nan" in x:
        features.remove(x)
        
features=features
features=features[2:]

train_features = join.loc['train'][features]
train_target = join.loc['train']['fault_severity']

test_features = join.loc['test'][features]

In [17]:
join.head()

fault_severity  id  location_location 1  location_location 10  \
sample                                                                    
train  0               1   1                    0                     0   
       1               0   5                    0                     0   
       2               1   6                    0                     0   
       3               0   8                    0                     0   
       4               0  13                    0                     0   

          location_location 100  location_location 1000  \
sample                                                    
train  0                      0                       0   
       1                      0                       0   
       2                      0                       0   
       3                      0                       0   
       4                      0                       0   

          location_location 1001  location_location 1002  \
sample                                                     
train  0                       0                       0   
       1                       0                       0   
       2                       0                       0   
       3                       0                       0   
       4                       0                       0   

          location_location 1003  location_location 1004        ...          \
sample                                                          ...           
train  0                       0                       0        ...           
       1                       0                       0        ...           
       2                       0                       0        ...           
       3                       0                       0        ...           
       4                       0                       0        ...           

          event_type_event_type 7  event_type_event_type 8  \
sample                                                       
train  0                        0                        0   
       1                        0                        0   
       2                        0                        0   
       3                        0                        0   
       4                        0                        0   

          event_type_event_type 9  event_type_nan  \
sample                                              
train  0                        0               0   
       1                        0               0   
       2                        0               0   
       3                        0               0   
       4                        0               0   

          severity_type_severity_type 1  severity_type_severity_type 2  \
sample                                                                   
train  0                              1                              0   
       1                              0                              1   
       2                              0                              1   
       3                              0                              1   
       4                              0                              1   

          severity_type_severity_type 3  severity_type_severity_type 4  \
sample                                                                   
train  0                              0                              0   
       1                              0                              0   
       2                              0                              0   
       3                              0                              0   
       4                              0                              0   

          severity_type_severity_type 5  severity_type_nan  
sample                                                      
train  0                              0                  0  
       1                              0                  0  
      

In [8]:

join.head(15)

'fault_severity'

In [13]:
import xgboost as xgb

sz=(train_features.values).shape
train = train_features.values[:int(sz[0] * 0.7), :]
test = train_features.values[int(sz[0] * 0.7):, :]

train_X = train
train_Y = train_target.values[:int(sz[0]*0.7)]


test_X = test
test_Y = train_target.values[int(sz[0] * 0.7):]

xg_train = xgb.DMatrix(train_X, label=train_Y, feature_names=train_features.columns)
xg_test = xgb.DMatrix(test_X, label=test_Y, feature_names=train_features.columns)

In [18]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 1
param['gamma']= 3
param['max_depth'] = 15
param['min_child_weight']=4
param['max_delta_step']=4

param['nthread'] = 3
#param['subsample']=1
param['num_class'] = 3
param['eval_metric']='mlogloss'

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 4
bst = xgb.train(param, xg_train,num_round,evals=watchlist);
# get prediction
pred = bst.predict( xg_test );

xgb.cv(param, xg_train, num_round, nfold=5,
       metrics=['mlogloss'], seed = 0)

[0]	train-mlogloss:0.653171	test-mlogloss:0.670353
[1]	train-mlogloss:0.578092	test-mlogloss:0.599657
[2]	train-mlogloss:0.553969	test-mlogloss:0.579556
[3]	train-mlogloss:0.534443	test-mlogloss:0.568602


,test-mlogloss-mean,test-mlogloss-std,train-mlogloss-mean,train-mlogloss-std
0,0.689880,0.008796,0.661276,0.004137
1,0.629395,0.013871,0.588353,0.002724
2,0.608096,0.015692,0.559519,0.003816
3,0.599923,0.016745,0.548225,0.005809


In [19]:
xgb.plot_importance(booster=bst)

In [ ]:
plt.show()

In [7]:
trloc=train['location'].unique()
teloc=test['location'].unique()
counter=0
total=0
for x in teloc:
    total=total+1
    if x in trloc:
        counter=counter+1
counter/total

0.8103946102021174

In [11]:
prior_df=join.loc['train'].groupby('id').mean()
prior_df=prior_df['fault_severity']

id
1        1
5        0
6        1
8        0
13       0
19       1
20       0
23       0
24       0
26       0
27       0
28       0
29       0
31       0
33       0
38       0
43       0
44       0
49       0
53       0
54       0
55       0
57       1
59       0
60       0
61       1
64       0
67       0
68       2
69       1
        ..
18473    0
18474    0
18480    0
18482    0
18484    1
18490    0
18492    0
18493    0
18498    0
18503    0
18506    0
18507    0
18508    0
18511    1
18520    0
18524    1
18525    2
18527    1
18528    2
18530    0
18533    0
18535    0
18536    0
18537    0
18538    1
18539    0
18542    0
18543    1
18548    0
18550    0
Name: fault_severity, dtype: float64

In [8]:
join.loc['train']['volume'][join.loc['train']['id']==9320]

3730    3.670196
Name: volume, dtype: float64

In [ ]:
xg

In [77]:
join.loc['test'].shape

(11171, 1588)

In [211]:
true_test=test_features.values
xg_true_test=xgb.DMatrix(true_test)
test_response=bst.predict(xg_true_test)

In [198]:
test_response

array([[ 0.83686632,  0.13276784,  0.0303659 ],
       [ 0.64375108,  0.28828788,  0.06796105],
       [ 0.87293285,  0.09292647,  0.03414068],
       ..., 
       [ 0.8772341 ,  0.095026  ,  0.02773988],
       [ 0.89589626,  0.07657157,  0.02753224],
       [ 0.95903444,  0.0267894 ,  0.01417619]], dtype=float32)

In [199]:
plt.hist(test_response)
plt.show()

In [78]:
test_response.shape

(11171, 3)

In [212]:
#save predictions to required format 
#make it binary?
out=pd.DataFrame({'id':[],'predict_0':[],'predict_1':[],'predict_2':[]})
out['id']=join.loc['test']['id']
out['predict_0']=test_response.T[0]
out['predict_1']=test_response.T[1]
out['predict_2']=test_response.T[2]
out=out[['id','predict_0','predict_1','predict_2']]
out.to_csv('predictions.csv',index=False)

In [120]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import RidgeClassifierCV, Perceptron, SGDClassifier, PassiveAggressiveClassifier
from sklearn.dummy import DummyClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.svm import LinearSVC
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn import grid_search

In [77]:
train_x, test_x, train_y, test_y = train_test_split(train_features, train_target, test_size=0.31, random_state=0)
def benchmark(clf):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = t.time()
    #clf=OneVsRestClassifier(clf) #may want to remove for testing
    clf.fit(train_x, train_y)
    train_time = t.time() - t0
    print("Training time: %0.3fs" % train_time)

    t0 = t.time()
    y_pred_single = clf.predict(test_x)
    if (hasattr(clf,'predict_proba')):
        y_pred_dist = clf.predict_proba(test_x)
        logloss=log_loss(test_y,y_pred_dist)
        print("LogLoss: %0.3fs" % logloss)
    test_time = t.time() - t0
    print("Prediction time:  %0.3fs" % test_time)

    acc_score = accuracy_score(test_y, y_pred_single)
    print("Accuracy:   %0.3f" % acc_score)
    
    #if (hasattr(clf,'predict_proba')):
       #scores = cross_val_score(clf,train_features,train_target, scoring='log_loss')
        #loss_score=scores.mean()
        #print("CV LogLoss: %0.2f (+/- %0.2f)" % (loss_score, scores.std() * 2))
    
    print(classification_report(test_y, y_pred_single))
    clf_descr = str(clf).split('(')[0]
    
    if (hasattr(clf,'predict_proba')):
        return clf_descr, acc_score, logloss, train_time, test_time
    else:
        return clf_descr, acc_score, train_time, test_time

In [79]:
results = []
for clf, name in (
        (RidgeClassifierCV(), "Ridge Classifier"),
        (RandomForestClassifier(n_estimators=300), "Random forest"),
        (AdaBoostClassifier(), "AdaBoost"),
        (BaggingClassifier(), "Bagging"),
        (QuadraticDiscriminantAnalysis(), "DiscriminatnAnalysis")):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf))

Ridge Classifier
________________________________________________________________________________
Training: 
RidgeClassifierCV(alphas=(0.1, 1.0, 10.0), class_weight=None, cv=None,
         fit_intercept=True, normalize=False, scoring=None)
Training time: 4.643s
Prediction time:  0.038s
Accuracy:   0.727
             precision    recall  f1-score   support

        0.0       0.79      0.91      0.84      1476
        1.0       0.59      0.35      0.44       593
        2.0       0.49      0.55      0.52       220

avg / total       0.71      0.73      0.71      2289

Random forest
________________________________________________________________________________
Training: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verb

C:\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:688: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


In [115]:
#test a classifier

# Fit the model to training data
#parameters={'n_estimators':[100,150]},'max_depth':[3]}
clf = OneVsRestClassifier(GradientBoostingClassifier(n_estimators=200,verbose=1,presort=True,max_depth=7),n_jobs=-2)#,min_samples_leaf=10,min_samples_split=5)
#clf=grid_search.GridSearchCV(gbc, parameters,n_jobs=-2)
benchmark(clf)

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=7, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200, presort=True,
              random_state=None, subsample=1.0, verbose=1,
              warm_start=False),
          n_jobs=-2)
Training time: 92.315s
Prediction time:  0.573s
Accuracy:   0.739
             precision    recall  f1-score   support

        0.0       0.80      0.89      0.84      1476
        1.0       0.61      0.42      0.50       593
        2.0       0.55      0.60      0.58       220

avg / total       0.73      0.74      0.73      2289



('OneVsRestClassifier',
 0.73918741808650068,
 0.57045593736126277,
 92.31480956077576,
 0.5728719234466553)

In [122]:
#test a classifier

# Fit the model to training data
#parameters={'n_estimators':[100,150]},'max_depth':[3]}
clf = OneVsRestClassifier(RandomForestClassifier(n_estimators=400,verbose=1,min_samples_split=10,\
                                                 oob_score=True,n_jobs=-2),n_jobs=-2)
#clf=grid_search.GridSearchCV(gbc, parameters,n_jobs=-2)
benchmark(clf)

________________________________________________________________________________
Training: 
OneVsRestClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-2,
            oob_score=True, random_state=None, verbose=1, warm_start=False),
          n_jobs=-2)
Training time: 31.099s

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.1s finished
[Parallel(n_jobs=3)]: Done  44 tasks    


Prediction time:  1.980s
Accuracy:   0.751
             precision    recall  f1-score   support

        0.0       0.80      0.90      0.85      1476
        1.0       0.63      0.44      0.52       593
        2.0       0.59      0.56      0.57       220

avg / total       0.74      0.75      0.74      2289



('OneVsRestClassifier',
 0.75141983398864132,
 0.54245802948043198,
 31.099143743515015,
 1.9801154136657715)

In [110]:
scores = cross_val_score(clf,train_features,train_target, scoring='log_loss',n_jobs=-2,verbose=True)
print("LogLoss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

LogLoss: -0.56 (+/- 0.01)


[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   44.6s finished


In [111]:
bestCLF=clf

In [112]:
bestCLF.fit(train_features,train_target)

[Parallel(n_jobs=-2)]: Done  44 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-2)]: Done 194 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-2)]: Done 400 out of 400 | elapsed:   10.2s finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=800,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-2,
            oob_score=True, random_state=None, verbose=1, warm_start=False)

In [113]:
test_response=bestCLF.predict_proba(test_features)

[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed:    0.3s
[Parallel(n_jobs=3)]: Done 400 out of 400 | elapsed:    0.7s finished


In [5]:
#transform cat labels into digits
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for column in join.columns:
    if column not in notCat:
        join[column]=le.fit_transform(join[column])
    
join.head()

fault_severity     id  location  event_type  severity_type  \
sample                                                                 
train  0               1  14121       148          26              1   
       1               1  14121       148          26              1   
       2               1  14121       148          27              1   
       3               1  14121       148          27              1   
       4               0   9320      1027          26              1   

          resource_type  log_feature  volume  
sample                                        
train  0              2          237      19  
       1              2          148      19  
       2              2          237      19  
       3              2          148      19  
       4              2          240     200